In [ ]:
import Bio
from Bio import SeqIO
from Bio import Entrez
from Bio import Phylo
from Bio import AlignIO
import tkinter
from tkinter import *
from tkinter import filedialog
import os
import pandas as pd

In [ ]:
# select excel file with accession numbers for nucleotide sequences (available in the supplementary materials of the paper)
# this is just a shortcut to typing out the complete file path

root = tkinter.Tk()
request_load_file = filedialog.askopenfile(initialdir=os.getcwd())
if request_load_file:
      filepath = os.path.abspath(request_load_file.name)
root.destroy()

filepath

In [ ]:
#  Creates a new folder called "fasta_files" in the same directory where your spreadsheet file is located

folder = '\\'.join(filepath.split('\\')[:-1])
if not os.path.exists(folder+'\\fasta_files'):
    os.makedirs(folder+'\\fasta_files')

In [ ]:
# creates a local copy of the spreadsheet that you selected

df_accession = pd.read_excel(filepath)
df_accession

In [ ]:
#  This cell will fetch fasta strings from the genbank database and save them in individual fasta files with deflines that match the nomenclature in the solaralo dogs study
#  Expect this cell to have a longer runtime due to repeated interaction with the database

fasta_list = list()
fasta_string = str()

#  Iterate over available accession codes in the spreadsheet object
for accession in df_accession[df_accession.columns[1]].dropna():
    try:
        
#  email is needed to identify yourself to the Genbank database
        Entrez.email = 'chad.m.reed1989@gmail.com'
    
#  'efetch' command retreives data from Genbank ("gb") nucleotide database
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        
#  Boolean statements are used to name the defline appropriately (based on figure1 in solarolo dogs study)
        if not df_accession['Breed'][df_accession['Accession Number*']==accession].values[0]=='-':
            record.description = df_accession['Breed'][df_accession['Accession Number*']==accession].values[0]
        else:
            record.description = df_accession['ID'][df_accession['Accession Number*']==accession].values[0]
            
#  this creates individual fasta files for each accession number with updated deflines
        SeqIO.write(record,f'{folder}\\fasta_files\\{record.description}.fasta',"fasta")
    
#  convert the record object to text and add to a local list
        fasta_list.append(SeqIO.FastaIO.as_fasta(record))

#  exception is made when the accession code is not valid, jumps to the next accession code
    except Exception:
        pass

In [ ]:
# this cell writes the strings in the 'fasta_list' to a single .txt file

with open(f"{folder}canis_rDNA.txt", "w") as text_file:
    text_file.write(fasta_string.join(fasta_list))

In [ ]:
#  this cell uses list comprehension to iterate over the fasta files that you just created and writes them into a single fasta file

SeqIO.write(
    
#  wicked awesome list comprehension iterates over an imported function
    [SeqIO.read(f'{folder}\\fasta_files\\{i}',format='fasta') for i in 
     os.listdir(f'{folder}\\fasta_files')], 
    
#  input directory of new combined fasta file
    f"{folder}\\canid_rDNA.fasta",
    format="fasta")